In [1]:
import csv
import numpy as np

pwm_data = "../test/pwm_edges.log" # format: time_ns,value
freq_data = "./freq_response.csv" # format: frequency;V(/Vout) (phase);V(/Vout) (gain);
out_name = "output.wav"

In [2]:
times = []
values = []
with open(pwm_data, 'r') as f:
    r = csv.reader(f)
    for t, v in r:
        times.append(float(t) * 1e-9)  # convert ns to s
        values.append(float(v))
times = np.array(times)
values = np.array(values)

duration = times[-1]

print(duration)

0.40000163000000005


In [3]:
from scipy.interpolate import make_interp_spline

Fs = 28835840 # output sample rate
N = int(duration * Fs)
t_uniform = np.linspace(0, duration, N)

# PWM is piecewise-constant, so use zero-order hold
# interp1d with 'previous' gives step function
interp = make_interp_spline(times, values, k=0)
x = interp(t_uniform)

print(x[:20])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [4]:
freq = []
gain_db = []
phase_deg = []

with open(freq_data, 'r') as f:
    r = csv.reader(f, delimiter=';')
    for f_hz, p, g, _ in r:
        freq.append(float(f_hz))
        phase_deg.append(float(p))
        gain_db.append(float(g))

freq = np.array(freq)
gain = 10 ** (np.array(gain_db) / 20.0)
phase = np.deg2rad(phase_deg)

In [5]:
f_fft = np.fft.rfftfreq(N, 1/Fs)

print(len(f_fft))

# Interpolate magnitude & phase separately
gain_interp = np.interp(f_fft, freq, gain)
phase_interp = np.interp(f_fft, freq, phase)

H_fft = gain_interp * np.exp(1j * phase_interp)

print(gain_interp[:20])

5767192
[0.09063585 0.21172037 0.3512564  0.42738514 0.46860956 0.49219909
 0.50661215 0.51594502 0.52228059 0.52676364 0.53003527 0.53250033
 0.53439727 0.53589098 0.53708123 0.53804783 0.53884362 0.5395038
 0.5400613  0.54053135]


In [6]:
# apply filter in frequency domain
X = np.fft.rfft(x)
Y = X * H_fft
y = np.fft.irfft(Y)

print(x[:20])
print(y[:20])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0.14037407 0.14012857 0.1399607  0.13971286 0.13951496 0.13926651
 0.13905651 0.13880916 0.13859417 0.13834959 0.13813373 0.13789352
 0.13767968 0.13744536 0.13723583 0.13700878 0.13680542 0.13658692
 0.1363913  0.13618247]


In [7]:
import wave
import array

WAV_SAMPLE_RATE = 48000

# interp Fs down to WAV_SAMPLE_RATE
num_wav_samples = int(len(y) * WAV_SAMPLE_RATE / Fs)
y = np.interp(np.linspace(0, len(y), num_wav_samples, endpoint=False),
              np.arange(len(y)),
              y)

y_norm = y / np.max(np.abs(y)) * 0.95
samples = [int(v * 32767) for v in y_norm]

# Write to WAV (16-bit PCM)
wavname = out_name.rsplit('.', 1)[0] + ".wav"
with wave.open(wavname, "wb") as f:
    f.setnchannels(1)
    f.setsampwidth(2) # 2 byte samples
    f.setframerate(WAV_SAMPLE_RATE)
    f.writeframes(array.array('h', samples).tobytes())